In [1]:
import pandas as pd
import glob
import os
from tqdm.auto import tqdm
import pycountry

In [2]:
code_to_name = {c.alpha_2: c.name for c in pycountry.countries}

In [3]:
path = '../raw_downloads/patents_view'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_list = []

for filename in tqdm(all_files):
    df = pd.read_csv(filename, index_col=None, header=0, low_memory=False)
    df_list.append(df)

combined_df = pd.concat(df_list, axis=0, ignore_index=True)

print(f"Loaded {len(all_files)} files.")
print(f"Total rows: {len(combined_df)}")

  0%|          | 0/25 [00:00<?, ?it/s]

Loaded 25 files.
Total rows: 7064526


In [4]:
df = combined_df.copy()

df = df.drop(columns=["d_inventor", "d_cpc", "wipo_field_ids", "n_wipo"]).query("application_number.notnull() and application_year.notnull()")
df[["n_cpc", "n_ipc"]] = (
    df[["n_cpc", "n_ipc"]]
    .fillna(0)
    .apply(pd.to_numeric, downcast="integer")
)
df = df.query(
    "n_cpc < 50 and n_ipc < 50"
)

df = df.astype({
    "patent_number": "string",
    "application_number": "string",
})

int_cols = [
    "assignee_ind",
    "grant_year",
    "application_year",
    "inventors",
    "team_size",
    "men_inventors",
    "women_inventors",
]
df[int_cols] = df[int_cols].apply(pd.to_numeric, downcast="integer")

df["assignee_sequence"] = pd.to_numeric(df["assignee_sequence"].fillna(-1), downcast="integer")


bool_cols = ["d_assignee", "d_location", "d_application", "d_ipc", "d_wipo"]
df[bool_cols] = df[bool_cols].astype(bool)

for i in range(1, 11):
    male_col = f"male_flag{i}"
    df[male_col] = (
        df[male_col]
        .map({1: "Male", 0: "Female"})
        .fillna("Gender not attributed")
        .astype("string")
    )

    inv_id_col = f"inventor_id{i}"
    inv_name_col = f"inventor_name{i}"
    placeholder = f"No inventor of {i}th rank"

    df[inv_id_col] = df[inv_id_col].fillna(placeholder).astype("string")
    df[inv_name_col] = df[inv_name_col].fillna(placeholder).astype("string")

cat_fill = {
    "country": "Unknown country",
    "county": "Unknown county",
    "state": "Unknown state",
    "city": "Unknown city",
    "assignee": "Unknown assignee",
    "first_wipo_sector_title": "Unknown sector",
    "first_wipo_field_title": "Unknown field",
}
for col, default in cat_fill.items():
    df[col] = df[col].fillna(default).astype("string")

df["country"] = (
    df["country"]
    .map(code_to_name)
    .fillna("Unknown country")
    .astype("string")
)

for col_prefix in ["ipc_", "cpc_"]:
    df[f"{col_prefix}sections_clean"] = df[f"{col_prefix}sections"].fillna("").str.strip().str.upper()
    dummies = (
        df[f"{col_prefix}sections_clean"]
        .str.get_dummies(sep=" ")
        .add_prefix(col_prefix)
    ).astype("bool")

    df = pd.concat([df, dummies], axis=1).drop(columns=[f"{col_prefix}sections_clean", f"{col_prefix}sections"])

In [5]:
df.to_parquet("../data/patents.parquet", compression="gzip", index=False)

In [1]:
import pandas as pd

df = pd.read_parquet("../data/patents.parquet")

In [5]:
df["assignee"].value_counts()

assignee
Unknown assignee                               564869
SAMSUNG ELECTRONICS CO., LTD.                  157760
INTERNATIONAL BUSINESS MACHINES CORPORATION    139209
CANON KABUSHIKI KAISHA                          69358
SONY GROUP CORPORATION                          50317
                                                ...  
CAUCUS LIMITED                                      1
IMPETUS AGRICULTURE, INC.                           1
FRUTAROM LIMITED                                    1
ANKA ANGEWANDTE KAFFEETECHNOLOGIE GMBH              1
UTILISCOPE CORPORATION                              1
Name: count, Length: 404324, dtype: Int64

In [3]:
df.shape

(7036574, 92)